In [14]:
import requests
import os
import pandas as pd
import json

In [17]:
pd.set_option('display.max_columns', None)

In [2]:
API_KEY = os.environ["YELP_FUSION_API_KEY"]

In [32]:
skytrain_stations = pd.read_csv('skytrain_stations.csv')
skytrain_sample = skytrain_stations.head(2)
skytrain_sample

,Name,Rating,Number of Ratings,Place ID,Latitude,Longitude
0,Broadway-City Hall Station,4.1,90,ChIJQx4neN1zhlQRdHFbwLLpxeM,49.262997,-123.114541
1,Vancouver City Centre,4.3,146,ChIJ04sXE39xhlQR1Woi1gmeQXY,49.282469,-123.118613


In [34]:
tempJson

{'businesses': [{'id': 'NdEPf2Ls5Ql3_nkwjqKvXA',
   'alias': 'the-flying-pig-vancouver-5',
   'name': 'The Flying Pig',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/8rMqIUUHHZRlFn2jctlSnw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/the-flying-pig-vancouver-5?adjust_creative=0_vDvM4vwmFucTxQib_w6Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=0_vDvM4vwmFucTxQib_w6Q',
   'review_count': 1281,
   'categories': [{'alias': 'newcanadian', 'title': 'Canadian (New)'},
    {'alias': 'seafood', 'title': 'Seafood'}],
   'rating': 4.0,
   'coordinates': {'latitude': 49.27501, 'longitude': -123.122},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '1168 Hamilton Street',
    'address2': 'Unit 104',
    'address3': '',
    'city': 'Vancouver',
    'zip_code': 'V6B 2S2',
    'country': 'CA',
    'state': 'BC',
    'display_address': ['1168 Hamilton Street',
     'Unit 104',
     'Vancouver, BC V6B 2S2',
     'Canada']},
 

In [39]:
url = "https://api.yelp.com/v3/businesses/search"
headers = {
    "accept": "application/json",
    "authorization": 'bearer %s' % API_KEY #This entire code is pretty much the same as the one I made for FourSquare with a couple of differences. 
          }

YelpVenues = pd.DataFrame(columns = ['Venue Name', 'Ratings', 'Review Count', 'Primary Category', 'SkyTrain Station', 'Distance to Station', 'Longitude', 'Latitude', 'ID', 'URL', 'Source']) 

for row in skytrain_sample.index:
    StationName = str(
        skytrain_sample['Name'][row]
    )
    Lat = str(
        skytrain_sample['Latitude'][row]
    )
    Long = str(
        skytrain_sample['Longitude'][row]
    )
    params = {
        "term": "Restaurants",
        "latitude": Lat, 
        "longitude": Long,
        "radius": 1000, 
        "sort":"DISTANCE",
        "limit": 50
    }
    
    response = requests.request(
        "GET", url, params=params, headers=headers
    )
    
    tempJson = json.loads(
        response.text
    )
    tempdb = pd.json_normalize(
        tempJson['businesses']
    )
    for venue in tempdb.index:
        VenueName = tempdb['name'][venue]
        Distance = tempdb['distance'][venue]
        Rating = tempdb['rating'][venue]
        ReviewCount = tempdb['review_count'][venue]
        PrimaryCategory = tempdb['categories'][venue][0]['title']
        Latitude = tempdb['coordinates.latitude'][venue]
        Longitude = tempdb['coordinates.longitude'][venue]
        YelpID = tempdb['id'][venue]
        VenueURL = tempdb['url'][venue]
        new_row = pd.Series({
            'Venue Name': VenueName, 
            'Ratings': Rating,
            'Review Count': ReviewCount,
            'Primary Category': PrimaryCategory,
            'SkyTrain Station': StationName, 
            'Distance to Station': Distance,
            'Longitude': Longitude,
            'Latitude': Latitude,
            'ID': YelpID,
            'URL': VenueURL,
            'Source': 'Yelp'
        } 
        )        
        YelpVenues = pd.concat(
            [YelpVenues, new_row.to_frame().T], 
            ignore_index = True
        )
    

    
YelpVenues.to_csv('yelp_venues.csv', index=False) 
YelpVenues

,Venue Name,Ratings,Review Count,Primary Category,SkyTrain Station,Distance to Station,Longitude,Latitude,ID,URL,Source
0,Seaport City Seafood Restaurant,4.5,19,Dim Sum,Broadway-City Hall Station,111.874634,-123.115152,49.263921,F5wxgIiZE7LYQxgqhI483A,https://www.yelp.com/biz/seaport-city-seafood-...,Yelp
1,Saku,4.5,213,Japanese,Broadway-City Hall Station,155.321784,-123.116675,49.263101,XAH2HpuUUtu7CUO26pbs4w,https://www.yelp.com/biz/saku-vancouver-8?adju...,Yelp
2,Kamei on Broadway,4.0,16,Japanese,Broadway-City Hall Station,270.910833,-123.118152,49.263643,ziQqb3wVR0WHCOyFiw2XmQ,https://www.yelp.com/biz/kamei-on-broadway-van...,Yelp
3,Hokkaido Ramen Santouka,4.0,222,Noodles,Broadway-City Hall Station,168.846607,-123.116892,49.263127,nkDZY5xqihF3XtZMzzfqqg,https://www.yelp.com/biz/hokkaido-ramen-santou...,Yelp
4,Lumiere Cafe,4.5,3,Cafes,Broadway-City Hall Station,108.113478,-123.115865,49.263468,sKqS0SUZRtLwAr2LA3W3eg,https://www.yelp.com/biz/lumiere-cafe-vancouve...,Yelp
...,...,...,...,...,...,...,...,...,...,...,...
95,Dingding Restaurant,4.0,3,Chinese,Vancouver City Centre,155.150198,-123.116568,49.282059,UYdZNpw-RyGiTe2bZN69Zg,https://www.yelp.com/biz/dingding-restaurant-v...,Yelp
96,Burgoo Bistro,4.0,169,Comfort Food,Vancouver City Centre,717.795282,-123.12763,49.27968,w4akEdaAoTT-bIgStOqeng,https://www.yelp.com/biz/burgoo-bistro-vancouv...,Yelp
97,The Roof,4.0,80,Seafood,Vancouver City Centre,339.375853,-123.122288,49.284358,NRItgoGiQEH1PyAgBmG6dQ,https://www.yelp.com/biz/the-roof-vancouver?ad...,Yelp
98,Mott 32,4.0,175,Chinese,Vancouver City Centre,650.677952,-123.124535,49.286864,wVlLeeowJn_VEhMOJVxMBQ,https://www.yelp.com/biz/mott-32-vancouver-2?a...,Yelp


In [ ]:
YelpVenues.

In [36]:
TempDB

,Venue Name,Ratings,Review Count,Primary Category,SkyTrain Station,Distance to Station,Longitude,Latitude,ID,URL,Source
0,Seaport City Seafood Restaurant,4.5,19,Dim Sum,Broadway-City Hall Station,111.874634,-123.115152,49.263921,F5wxgIiZE7LYQxgqhI483A,https://www.yelp.com/biz/seaport-city-seafood-...,Yelp
1,Saku,4.5,213,Japanese,Broadway-City Hall Station,155.321784,-123.116675,49.263101,XAH2HpuUUtu7CUO26pbs4w,https://www.yelp.com/biz/saku-vancouver-8?adju...,Yelp
2,Kamei on Broadway,4.0,16,Japanese,Broadway-City Hall Station,270.910833,-123.118152,49.263643,ziQqb3wVR0WHCOyFiw2XmQ,https://www.yelp.com/biz/kamei-on-broadway-van...,Yelp
3,Hokkaido Ramen Santouka,4.0,222,Noodles,Broadway-City Hall Station,168.846607,-123.116892,49.263127,nkDZY5xqihF3XtZMzzfqqg,https://www.yelp.com/biz/hokkaido-ramen-santou...,Yelp
4,Lumiere Cafe,4.5,3,Cafes,Broadway-City Hall Station,108.113478,-123.115865,49.263468,sKqS0SUZRtLwAr2LA3W3eg,https://www.yelp.com/biz/lumiere-cafe-vancouve...,Yelp
5,La Taqueria Pinche Taco Shop,4.0,681,Mexican,Broadway-City Hall Station,145.071926,-123.112736,49.263559,6iOAgzJ0DRZNSKA3FSrrOg,https://www.yelp.com/biz/la-taqueria-pinche-ta...,Yelp
6,Heritage Asian Eatery,4.0,87,Pan Asian,Broadway-City Hall Station,155.526827,-123.11247,49.26287,gdSAYWUI2MTgrk08r_OZSg,https://www.yelp.com/biz/heritage-asian-eatery...,Yelp
7,Marulilu Cafe,4.0,285,Cafes,Broadway-City Hall Station,53.388446,-123.11415,49.26338,NensKn1MSVU_rm-1Y6WlFA,https://www.yelp.com/biz/marulilu-cafe-vancouv...,Yelp
8,Alley 16,5.0,1,Barbeque,Broadway-City Hall Station,664.438725,-123.11474,49.257017,Rb0S3Ep3flHbPDSb0uXK4w,https://www.yelp.com/biz/alley-16-vancouver?ad...,Yelp
9,A.Bento,4.0,24,Taiwanese,Broadway-City Hall Station,313.763061,-123.118857,49.263164,y583PSvm5c6tZYmRiboYYA,https://www.yelp.com/biz/a-bento-vancouver?adj...,Yelp
